In [1]:
import networkx as nx
import pandas as pd
from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Import data 
data = pd.read_csv('../Notebooks/datasets/timeSeries1_verified.csv',header=0, index_col=0)
df = pd.DataFrame(data)

df.head(10)

,identifier,experiment,type,_time,count
0,0,391.0,2.0,2023-10-04 19:25:30+00:00,1574
1,0,391.0,2.0,2023-10-04 19:25:40+00:00,6412
2,0,391.0,2.0,2023-10-04 19:25:50+00:00,4602
3,0,391.0,2.0,2023-10-04 19:26:00+00:00,4605
4,0,391.0,2.0,2023-10-04 19:26:10+00:00,6171
5,0,391.0,2.0,2023-10-04 19:26:20+00:00,4642
6,0,391.0,2.0,2023-10-04 19:26:30+00:00,4643
7,0,391.0,2.0,2023-10-04 19:26:40+00:00,6142
8,0,391.0,2.0,2023-10-04 19:26:50+00:00,5016
9,0,391.0,2.0,2023-10-04 19:27:00+00:00,4996


In [3]:
# Transpose the table -> type and count

df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['identifier'],df['type'],pd.Grouper(key="_time", freq='1s')])['count'].sum().reset_index()
# by_time.head(10)
pivoted = pd.pivot_table(by_time,index=["identifier","experiment", pd.Grouper(freq="10s", key="_time")], columns="type", values="count", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_25591/1443210762.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,identifier,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,0,391.0,2023-10-04 19:25:30+00:00,1574.0,192.0,0.0,0.0,0.0,109.0
1,0,391.0,2023-10-04 19:25:40+00:00,6412.0,795.0,0.0,0.0,0.0,395.0
2,0,391.0,2023-10-04 19:25:50+00:00,4602.0,576.0,0.0,0.0,1.0,361.0
3,0,391.0,2023-10-04 19:26:00+00:00,4605.0,575.0,0.0,0.0,0.0,325.0
4,0,391.0,2023-10-04 19:26:10+00:00,6171.0,767.0,0.0,0.0,0.0,387.0
...,...,...,...,...,...,...,...,...,...
95,0,391.0,2023-10-04 19:44:50+00:00,19744.0,2935.0,0.0,0.0,0.0,1142.0
96,0,391.0,2023-10-04 19:45:00+00:00,15108.0,2238.0,0.0,0.0,0.0,987.0
97,0,391.0,2023-10-04 19:45:10+00:00,14310.0,2111.0,0.0,0.0,0.0,1136.0
98,0,391.0,2023-10-04 19:45:20+00:00,19631.0,2898.0,0.0,0.0,0.0,1029.0


In [4]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageReceived', 3.0: 'messageDuplicated', 11.0: 'graft', 12.0: 'prune', 16.0: 'iwant', 17.0:'ihave'})
eventsTS.head(10)

,identifier,experiment,_time,messageReceived,messageDuplicated,graft,prune,iwant,ihave
0,0,391.0,2023-10-04 19:25:30+00:00,1574.0,192.0,0.0,0.0,0.0,109.0
1,0,391.0,2023-10-04 19:25:40+00:00,6412.0,795.0,0.0,0.0,0.0,395.0
2,0,391.0,2023-10-04 19:25:50+00:00,4602.0,576.0,0.0,0.0,1.0,361.0
3,0,391.0,2023-10-04 19:26:00+00:00,4605.0,575.0,0.0,0.0,0.0,325.0
4,0,391.0,2023-10-04 19:26:10+00:00,6171.0,767.0,0.0,0.0,0.0,387.0
5,0,391.0,2023-10-04 19:26:20+00:00,4642.0,575.0,0.0,0.0,0.0,372.0
6,0,391.0,2023-10-04 19:26:30+00:00,4643.0,575.0,0.0,0.0,0.0,326.0
7,0,391.0,2023-10-04 19:26:40+00:00,6142.0,763.0,0.0,0.0,0.0,386.0
8,0,391.0,2023-10-04 19:26:50+00:00,5016.0,616.0,0.0,0.0,0.0,369.0
9,0,391.0,2023-10-04 19:27:00+00:00,4996.0,620.0,0.0,0.0,0.0,326.0


In [ ]:
#Bandwidth
deliverMessage=df.loc[